<a href="
https://colab.research.google.com/drive/1uhuvoJViyXdhckE7EYHNSXchm9nzAMU2?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>



In [1]:
 !pip install scanpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 15.1 MB/s 
     |████████████████████████████████| 96 kB 2.9 MB/s 
     |████████████████████████████████| 11.2 MB 58.7 MB/s 
     |████████████████████████████████| 88 kB 4.7 MB/s 
     |████████████████████████████████| 965 kB 43.9 MB/s 
     |████████████████████████████████| 1.1 MB 51.9 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=da3369b08296118e07a6101524c05391d6c1a81bed4ae3300d52aac00754c179
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55512 sha256=b3556e615c79bdbafc440c174500d1b84b58c9b97f3393c965d8fd4c0d53b6b1
  Stored in directory: /root/.cache/pip/wheels/19/bc/eb/974072a56a7082a302f

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir /content/drive/MyDrive/scintegration/checkpoints

mkdir: cannot create directory ‘/content/drive/MyDrive/scintegration/checkpoints’: File exists


In [5]:
import torch
from torch import nn, optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import scanpy as sc
import plotly.express as px
import plotly.io as pio
import sklearn.preprocessing


pio.renderers.default = "notebook"

In [6]:
import platform
platform.platform()
if torch.cuda.is_available:
     device = torch.device("cuda:0")

In [7]:
torch.cuda.is_available()

True

In [8]:
scdata = sc.read_h5ad("/content/drive/MyDrive/scintegration/GEX.h5ad")

In [9]:
scdata.shape

(69249, 13431)

In [10]:
# sc.pp.log1p(scdata)

In [11]:
class GEX_Dataset(torch.utils.data.Dataset):
    
      def __init__(self, data,  scaler = None, cat_var = None):
          
            self.data = data
            
            # we need to work with the dense matrix
            self.values = data.X.todense()
            
            self.cat_var = cat_var
            
            # numerically encode the labels
            self.cat_var_data =  torch.tensor(sklearn.preprocessing.LabelEncoder().fit_transform(self.data.obs[self.cat_var]))
            
            # scale the data according to user inpt to scaler argument
            if scaler == "Standard":
                self.scaled_values = torch.tensor(sklearn.preprocessing.StandardScaler().fit_transform(self.values))
            elif scaler == "MinMax":
                self.scaled_values = torch.tensor(sklearn.preprocessing.MinMaxScaler().fit_transform(self.values))
            else:
                self.scaled_values = torch.tensor(self.values)
                
    #   return the number of genes when called 
             
      @property
      def n_features(self):
          return self.values.shape[1]
          
          
    #  A dataset class needs the following two methods to work with the dataloader class     
          
    #   return the number of cells when called
      def __len__(self):
          return len(self.data)
    
    #  return an individual cell and its label when called
      def __getitem__(self, idx):
           return self.scaled_values[idx], self.cat_var_data[idx]

    

In [12]:
GEX_Dataset = GEX_Dataset(scdata, scaler = "Standard", cat_var = "batch")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning:

np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning:

np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html



In [13]:
input_size = GEX_Dataset.n_features
hidden_size = 128 # hidden dimension
latent_size = 16  # latent vector dimension

batch_size = 128
epochs = 20
lr = 1e-4
train_loss = []
log_interval = 20
checkpoints = "/content/drive/MyDrive/scintegration/checkpoints"

In [14]:
GEX_Dataset.n_features

13431

In [15]:
GEX_dataloader = DataLoader(GEX_Dataset, batch_size=batch_size, shuffle=True)
GEX_dataloader_test = DataLoader(GEX_Dataset, batch_size=batch_size, shuffle=False)

In [16]:
class VAE(nn.Module):
    def __init__(self) :
        super(VAE, self).__init__()
        self.efc1 = nn.Linear(input_size, hidden_size)
        self.efc2 = nn.Linear(hidden_size, hidden_size)
        self.efc3_mu = nn.Linear(hidden_size, latent_size)
        self.efc3_sigma = nn.Linear(hidden_size, latent_size)
        
        self.dfc1 = nn.Linear(latent_size, hidden_size)
        self.dfc2 = nn.Linear(hidden_size, hidden_size)
        self.dfc3 = nn.Linear(hidden_size, input_size)
        
    def encode(self, x):
        x = self.efc1(x)
        x = F.relu(x)
        x = self.efc2(x)
        x = F.relu(x)
        mu = self.efc3_mu(x)
        sigma = self.efc3_sigma(x)
        return((mu,sigma))
    
    def reparametarise (self, mu, sigma):
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return(mu + eps*std)
    
    def decode(self, x):
        x = self.dfc1(x)
        x = F.relu(x)
        x = self.dfc2(x)
        x = F.relu(x)
        x = self.dfc3(x)
        x = torch.tanh(x)
        return(x)
    
    def forward(self, x):
        mu, sigma = self.encode(x)
        z = self.reparametarise(mu, sigma)
        x_gen = self.decode(z)
        return(x_gen, mu, sigma)
      

       

In [17]:
def loss_function(x_gen, x, mu, sigma):
    BCE = F.binary_cross_entropy_with_logits(x_gen, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - torch.exp(sigma))
    return(BCE + KLD)

In [18]:
def init_layers(layer):
    if type(layer) == nn.Linear:
        torch.nn.init.uniform_(layer.weight -0.08, 0.08)

In [19]:
vae = VAE()
vae.to(device)
vae = vae.apply(init_layers)
optimizer = optim.Adam(vae.parameters(), lr=lr)
vae.train()

VAE(
  (efc1): Linear(in_features=13431, out_features=128, bias=True)
  (efc2): Linear(in_features=128, out_features=128, bias=True)
  (efc3_mu): Linear(in_features=128, out_features=16, bias=True)
  (efc3_sigma): Linear(in_features=128, out_features=16, bias=True)
  (dfc1): Linear(in_features=16, out_features=128, bias=True)
  (dfc2): Linear(in_features=128, out_features=128, bias=True)
  (dfc3): Linear(in_features=128, out_features=13431, bias=True)
)

In [ ]:

for epoch in range(epochs+1):
	for batch_idx, (data, _) in enumerate(GEX_dataloader):
		data = data.to(device)
		optimizer.zero_grad()
		x_gen, mu, sigma = vae(data)
		loss = loss_function(x_gen, data, mu, sigma)
		if torch.isnan(loss):
			print(f"Nan Loss @ Epoch {epoch} Batch {batch_idx}")
			break
		loss.backward()
		optimizer.step()
		train_loss.append(loss.item())
		if batch_idx % log_interval == 0:
			print(f"Train epoch {epoch} [{batch_idx*len(data)}/{len(GEX_dataloader.dataset)}] Loss: {loss.item():.6f}")
		
		torch.save(vae.state_dict(), os.path.join(checkpoints, f'vae_epoch_{epoch}.pt'))
	





Train epoch 0 [0/69249] Loss: 1194281.125000
Train epoch 0 [2560/69249] Loss: 1168533.375000
Train epoch 0 [5120/69249] Loss: 1098465.250000
Train epoch 0 [7680/69249] Loss: 910684.687500
Train epoch 0 [10240/69249] Loss: 691564.250000
Train epoch 0 [12800/69249] Loss: 593120.062500
Train epoch 0 [15360/69249] Loss: 562120.812500
Train epoch 0 [17920/69249] Loss: 572966.062500
Train epoch 0 [20480/69249] Loss: 550089.625000
Train epoch 0 [23040/69249] Loss: 561971.000000
Train epoch 0 [25600/69249] Loss: 559031.750000
Train epoch 0 [28160/69249] Loss: 556502.375000
Train epoch 0 [30720/69249] Loss: 531279.250000
Train epoch 0 [33280/69249] Loss: 559358.500000
Train epoch 0 [35840/69249] Loss: 537757.750000
Train epoch 0 [38400/69249] Loss: 537992.375000
Train epoch 0 [40960/69249] Loss: 532498.625000
Train epoch 0 [43520/69249] Loss: 551549.312500
Train epoch 0 [46080/69249] Loss: 532538.687500
Train epoch 0 [48640/69249] Loss: 532012.375000
Train epoch 0 [51200/69249] Loss: 552131.562

In [ ]:
def load_checkpoint(model, optimizer, filename):
    print(f'Loading checkpoint {filename} for model {model}')
    model.load_state_dict(torch.load(filename))
    return model

In [ ]:
load_checkpoint(vae, optimizer, os.path.join(checkpoints, 'vae_epoch_4.pt'))
vae.eval()

In [ ]:
def get_embeddings(vae, dataloader):
    with torch.no_grad():
        embeddings = []
        labels = []
        for batch, label in dataloader:
            batch = batch.to(device)
            _, mu, _ = vae(batch)
            embeddings.append(mu.cpu().numpy())
            labels.append(label.cpu().numpy())
        embeddings = np.concatenate(embeddings)
        labels = np.concatenate(labels)
        return embeddings, labels

In [ ]:
embeddings, labels = get_embeddings(vae, GEX_dataloader_test)


In [ ]:
labels

In [ ]:
def plot_embeddings(dataframe, color_by=None, title=None):
    fig = px.scatter(dataframe, x=0, y=1, color=color_by, title=title)
    fig.show()
    

In [ ]:
dataframe = pd.DataFrame(embeddings, index = labels)

In [ ]:
fig = px.scatter(scdata.obsm["GEX_X_pca"], x=0, y=1, color=scdata.obs["batch"], template="simple_white")
fig = fig.update_layout(
    autosize=False,
    width=1400,
    height=1000)
fig.show()


In [ ]:
plot_embeddings(dataframe, color_by=dataframe.index, title="VAE Embeddings")

In [ ]:
scdata.obsm["VAE_z"] = embeddings

In [ ]:
scdata.obsm["VAE_z"].shape

In [ ]:
sc.pp.neighbors(scdata, use_rep="VAE_z")

In [ ]:
sc.tl.umap(scdata, min_dist=0.3)

In [ ]:
scdata

In [ ]:
sc.pl.umap(scdata, color="cell_type", wspace=0.5)

In [ ]:
sc.pl.umap(scdata, color=["batch", "cell_type"], wspace=0.5)

In [ ]:
def decode_embeddings(vae, embeddings):
    with torch.no_grad():
        embeddings = torch.tensor(embeddings).to(device)
        x_gen = vae.decode(embeddings)
        return x_gen.cpu().numpy()

In [ ]:
scdata.obsm["VAE_x"] = decode_embeddings(vae, embeddings)

In [ ]:
sc.pp.neighbors(scdata, use_rep="VAE_x", n_neighbors=30)

In [ ]:
sc.tl.umap(scdata, min_dist=0.3)

In [ ]:
sc.pl.umap(scdata, color="batch")